# FloPy Model Grids - Geoprocessing

## Notebook Setup

In [ ]:
import os
import sys
import shutil
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from shapely.geometry import Polygon, LineString
import flopy
from flopy.discretization import StructuredGrid
import flopy.plot.styles as styles

In [ ]:
# import all plot style information from defaults.py
from defaults import *

In [ ]:
from model_splitter import Mf6Splitter

In [ ]:
fine_topo = flopy.utils.Raster.load("./grid_data/fine_topo.asc")

In [ ]:
fine_topo.plot()

In [ ]:
Lx = 180000
Ly = 100000
extent = (0, Lx, 0, Ly)
dv0 = 5.0
levels = np.arange(10, 110, 10)
vmin, vmax = 0.0, 100.0

In [ ]:
temp_path = "./temp"
if not os.path.isdir(temp_path):
    os.mkdir(temp_path)

In [ ]:
figwidth = 180  # 90 # mm
figwidth = figwidth / 10 / 2.54  # inches
figheight = figwidth
figsize = (figwidth, figheight)

# Basin Example

In [ ]:
boundary_polygon = string2geom(boundary)
print("len boundary", len(boundary_polygon))
bp = np.array(boundary_polygon)

sgs = [
    string2geom(sg) for sg in (streamseg1, streamseg2, streamseg3, streamseg4)
]

fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot()
ax.set_aspect("equal")

riv_colors = ("blue", "cyan", "green", "orange", "red")

ax.plot(bp[:, 0], bp[:, 1], "ro-")
for idx, sg in enumerate(sgs):
    print("Len segment: ", len(sg))
    sa = np.array(sg)
    ax.plot(sa[:, 0], sa[:, 1], color=riv_colors[idx], lw=0.75, marker="o")

In [ ]:
def set_idomain(grid, boundary):
    from flopy.utils.gridintersect import GridIntersect
    from shapely.geometry import Polygon

    ix = GridIntersect(grid, method="vertex", rtree=True)
    result = ix.intersect(Polygon(boundary))
    idx = [coords for coords in result.cellids]
    idx = np.array(idx, dtype=int)
    nr = idx.shape[0]
    if idx.ndim == 1:
        idx = idx.reshape((nr, 1))
    print(idx.shape, idx.ndim)
    idx = tuple([idx[:, i] for i in range(idx.shape[1])])
    # idx = (idx[:, 0], idx[:, 1])
    idomain = np.zeros(grid.shape[1:], dtype=int)
    idomain[idx] = 1
    idomain = idomain.reshape(grid.shape)
    grid.idomain = idomain

In [ ]:
# Create a regular MODFLOW grid
Lx = 180000
Ly = 100000
dx = dy = 5000 #166.666666667
nlay = 1
nrow = int(Ly / dy) + 1
ncol = int(Lx / dx) + 1
print(Lx, Ly, nrow, ncol)
delr = np.array(ncol * [dx])
delc = np.array(nrow * [dy])
top = np.ones((nrow, ncol)) * 1000.0
botm = np.ones((nlay, nrow, ncol)) * -100.0
regular_grid = StructuredGrid(
    nlay=nlay, delr=delr, delc=delc, xoff=0.0, yoff=0.0, top=top, botm=botm
)

set_idomain(regular_grid, boundary_polygon)

In [ ]:
top_rg = fine_topo.resample_to_grid(
    regular_grid,
    band=fine_topo.bands[0],
    method="linear",
    extrapolate_edges=True,
)

In [ ]:
regular_grid.shape

In [ ]:
ixs = flopy.utils.GridIntersect(regular_grid, method="structured")
cellids = []
for sg in sgs:
    v = ixs.intersect(LineString(sg), sort_by_cellid=True)
    cellids += v["cellids"].tolist()
intersection_rg = np.zeros(regular_grid.shape[1:])
for loc in cellids:
    intersection_rg[loc] = 1

In [ ]:
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot()
pmv = flopy.plot.PlotMapView(modelgrid=regular_grid)
ax.set_aspect("equal")
pmv.plot_array(top_rg)
pmv.plot_array(
    intersection_rg,
    masked_values=[
        0,
    ],
    alpha=0.2,
    cmap="Reds_r",
)
pmv.plot_grid(lw=0.25, color="0.5")
pmv.plot_inactive()
#regular_grid.plot(ax=ax, )
ax.plot(bp[:, 0], bp[:, 1], "r-")
for sg in sgs:
    sa = np.array(sg)
    ax.plot(sa[:, 0], sa[:, 1], "b-")

### Build a model

In [ ]:
newton = True

# redefine the number of layers
nlay = 5

In [ ]:
if newton:
    newton_options = "NEWTON UNDER_RELAXATION"
    linear_accelerator = "bicgstab"
    outer_maximum = 1000
    rewet_record = None
    wetdry = None
else:
    newton_options = None
    linear_accelerator = "cg"
    outer_maximum = 10000
    rewet_record = "WETFCT 1.0 IWETIT 1 IHDWET 1"
    wetdry = -0.001

In [ ]:
drn_intersection = []
drn_cellids = []
drn_lengths = []
for sg in sgs:
    v = ixs.intersect(LineString(sg), sort_by_cellid=True)
    drn_intersection.append(v)
    drn_cellids += v["cellids"].tolist()
    drn_lengths += v["lengths"].tolist()

In [ ]:
leakance = 1.0 / (0.5 * dv0)  # kv / b
drn_data = []
for (r, c), length in zip(drn_cellids, drn_lengths):
    x = regular_grid.xcellcenters[r, c]
    width = 5.0 + (14.0 / Lx) * (Lx - x)
    conductance = leakance * length * width
    drn_data.append((0, r, c, top_rg[r, c], conductance))
drn_data[:10]

In [ ]:
# groundwater discharge to surface
idomain = regular_grid.idomain[0].copy()
index = tuple(np.array(list(zip(*drn_cellids))))
idomain[index] = -1
gw_discharge_data = []
for r in range(nrow):
    for c in range(ncol):
        if idomain[r, c] < 1:
            continue
        conductance = leakance * dx * dy
        gw_discharge_data.append(
            (0, r, c, top_rg[r, c] - 0.5, conductance, 1.0)
        )
gw_discharge_data[:10]

In [ ]:
plt.imshow(idomain)

In [ ]:
topc = np.zeros((nlay, nrow, ncol), dtype=float)
botm = np.zeros((nlay, nrow, ncol), dtype=float)
dv = dv0
topc[0] = top_rg.copy()
botm[0] = topc[0] - dv
for idx in range(1, nlay):
    dv *= 1.5
    topc[idx] = botm[idx - 1]
    botm[idx] = topc[idx] - dv

In [ ]:
for k in range(nlay):
    print((topc[k] - botm[k]).mean())

In [ ]:
idomain = np.array([regular_grid.idomain[0, :, :].copy() for k in range(nlay)], dtype=int)
strt = np.array([top_rg.copy() for k in range(nlay)], dtype=float)

In [ ]:
sim = flopy.mf6.MFSimulation(
    sim_name="basin",
    sim_ws="basin",
    exe_name="mf6",
)
tdis = flopy.mf6.ModflowTdis(sim)
outer_head_close = 1e-6
inner_head_close = 0.1*outer_head_close
ims = flopy.mf6.ModflowIms(
    sim, linear_acceleration=linear_accelerator, complexity="simple", print_option="SUMMARY",
    outer_maximum=outer_maximum, inner_maximum=100, 
    outer_dvclose=outer_head_close, inner_dvclose=inner_head_close,
)
gwf = flopy.mf6.ModflowGwf(
    sim, save_flows=True, newtonoptions=newton_options
)

dis = flopy.mf6.ModflowGwfdis(
    gwf,
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=dx,
    delc=dy,
    idomain=idomain,
    top=top_rg,
    botm=botm,
    xorigin=0.0,
    yorigin=0.0,
)

ic = flopy.mf6.ModflowGwfic(gwf, strt=100.0)
npf = flopy.mf6.ModflowGwfnpf(
    gwf,
    save_specific_discharge=True,
    icelltype=0,
    k=1.0,
    rewet_record=rewet_record,
    wetdry=wetdry,
)
# sto = flopy.mf6.ModflowGwfsto(
#     gwf,
#     iconvert=1,
#     ss=1e-5,
#     sy=0.2,
#     steady_state=True,
# )
rch = flopy.mf6.ModflowGwfrcha(
    gwf,
    recharge=0.000001,
)
drn = flopy.mf6.ModflowGwfdrn(
    gwf,
    stress_period_data=drn_data,
    pname="river",
)
drn_gwd = flopy.mf6.ModflowGwfdrn(
    gwf,
    auxiliary=["depth"],
    auxdepthname="depth",
    stress_period_data=gw_discharge_data,
    pname="gwd",
)
oc = flopy.mf6.ModflowGwfoc(
    gwf,
    head_filerecord=f"{gwf.name}.hds",
    budget_filerecord=f"{gwf.name}.cbc",
    saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
    printrecord=[("BUDGET", "ALL")],
)

In [ ]:
sim.write_simulation()
sim.run_simulation()

### Plot the heads for the full model

In [ ]:
gwfhead = flopy.utils.postprocessing.get_water_table(gwf.output.head().get_data())
hmin, hmax = gwfhead.min(), gwfhead.max()
contours = np.arange(0, 100, 10)
hmin, hmax

In [ ]:
def plot_head2d(heads):
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot()
    ax.set_xlim(0, Lx)
    ax.set_ylim(0, Ly)
    ax.set_aspect("equal")
    pmv = flopy.plot.PlotMapView(modelgrid=gwf.modelgrid, ax=ax)
    h = pmv.plot_array(heads, vmin=hmin, vmax=hmax)
    c = pmv.contour_array(heads, levels=contours, colors="white", linewidths=0.75, linestyles=":")
    plt.clabel(c, fontsize=8)
    pmv.plot_inactive()
    plt.colorbar(h, ax=ax, shrink=0.35)

    ax.plot(bp[:, 0], bp[:, 1], "r-")
    for sg in sgs:
        sa = np.array(sg)
        ax.plot(sa[:, 0], sa[:, 1], "b-")

In [ ]:
plot_head2d(gwfhead)

### Build a splitting array

In [ ]:
nrow_blocks, ncol_blocks = 1, 4

In [ ]:
row_inc, col_inc = int(nrow/nrow_blocks), int(ncol/ncol_blocks)
row_inc, col_inc

In [ ]:
icnt = 0
row_blocks = [icnt]
for i in range(nrow_blocks):
    icnt += row_inc
    row_blocks.append(icnt)
if row_blocks[-1] < nrow:
    row_blocks[-1] = nrow
row_blocks

In [ ]:
icnt = 0
col_blocks = [icnt]
for i in range(ncol_blocks):
    icnt += col_inc
    col_blocks.append(icnt)
if col_blocks[-1] < ncol:
    col_blocks[-1] = ncol
col_blocks

In [ ]:
mask = np.zeros((nrow, ncol), dtype=int)

In [ ]:
model_llcorner = {}
ival = 1
for idx in range(len(row_blocks)-1):
    for jdx in range(len(col_blocks)-1):
        mask[row_blocks[idx]:row_blocks[idx+1], col_blocks[jdx]:col_blocks[jdx+1]] = ival
        # get xoff yoff
        xoff = gwf.modelgrid.xvertices[row_blocks[idx+1], col_blocks[jdx]]
        yoff = gwf.modelgrid.yvertices[row_blocks[idx+1], col_blocks[jdx]]
        model_llcorner[ival] = (xoff, yoff)
        # increment model number
        ival += 1

In [ ]:
model_llcorner

In [ ]:
np.unique(mask)

In [ ]:
plt.imshow(mask)

### Split into (nrow_blocks, ncol_blocks) models

In [ ]:
mfsplit = Mf6Splitter(sim)

In [ ]:
new_sim = mfsplit.split_model(mask)

In [ ]:
new_ws = "basin_split"
shutil.rmtree(new_ws, ignore_errors=True) 
new_sim.set_sim_path(new_ws)

### Plot the submodels

In [ ]:
extent = (0., Lx, 0., Ly)

In [ ]:
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot()
ax.set_xlim(0, Lx)
ax.set_ylim(0, Ly)
ax.set_aspect("equal")
for name in new_sim.model_names:
    m = new_sim.get_model(name)
    pmv = flopy.plot.PlotMapView(modelgrid=m.modelgrid, ax=ax, extent=extent)
    pmv.plot_array(m.dis.top.array, vmin=top_rg.min(), vmax=top_rg.max())
    pmv.plot_inactive()
ax.plot(bp[:, 0], bp[:, 1], "r-")
for sg in sgs:
    sa = np.array(sg)
    ax.plot(sa[:, 0], sa[:, 1], "b-")

### Write and run the simulation 

In [ ]:
new_sim.write_simulation()

In [ ]:
new_sim.run_simulation()

### Plot the multi-model heads

In [ ]:
# build a single head dataset
model_names = list(new_sim.model_names)
head_tot_serial = np.zeros((nlay, nrow, ncol), dtype=float)
im = 0
for idx in range(len(row_blocks)-1):
    for jdx in range(len(col_blocks)-1):
        m = new_sim.get_model(model_names[im])
        mhead = m.output.head().get_data()
        head_tot_serial[:, row_blocks[idx]:row_blocks[idx+1], col_blocks[jdx]:col_blocks[jdx+1]] = mhead[:, :, :]
        # increment model number
        im += 1
wt_tot = flopy.utils.postprocessing.get_water_table(head_tot_serial)

In [ ]:
plot_head2d(wt_tot)

### Check: multi-model versus single model

In [ ]:
def plot_diff(diff, title):
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot()
    ax.set_xlim(0, Lx)
    ax.set_ylim(0, Ly)
    ax.set_aspect("equal")
        
    pmv = flopy.plot.PlotMapView(modelgrid=gwf.modelgrid, ax=ax, extent=extent)
    h = pmv.plot_array(diff)
    pmv.plot_inactive()
    plt.colorbar(h, ax=ax, shrink=0.35)
    plt.title(title)

In [ ]:
diff = head_tot_serial - gwfhead
ilay = 1
title = f"multi-model (serial) minus single model, layer {ilay+1}"
plot_diff(diff[ilay], title)

### Write and run parallel simulation

In [ ]:
par_ws = "basin_split_parallel"
shutil.rmtree(par_ws, ignore_errors=True) 
new_sim.set_sim_path(par_ws)
for idx, name in enumerate(new_sim.model_names):
    m = new_sim.get_model(name)
new_sim.write_simulation()

In [ ]:
ncpus = 4

In [ ]:
# write petsc cfg for parallel solve
petsc_db_file = os.path.join(par_ws, ".petscrc")
with open(petsc_db_file, 'w') as petsc_file:
    if ncpus > 1:
        petsc_file.write("-ksp_type bcgs\n")
        petsc_file.write("-pc_type bjacobi\n")
        petsc_file.write("-sub_pc_type ilu\n")
        petsc_file.write("-sub_pc_factor_levels 2\n")
        petsc_file.write("-dvclose 1.0e-6\n")
        petsc_file.write("-options_left no\n")
    else:
        petsc_file.write("-ksp_type bcgs\n")
        petsc_file.write("-pc_type ilu\n")
        petsc_file.write("-pc_factor_levels 2\n")
        petsc_file.write("-dvclose 1.0e-6\n")
        petsc_file.write("-options_left no\n")

In [ ]:
cmd = f"mpiexec --oversubscribe -np {ncpus} mf6 -p"
current_wd = os.getcwd()
os.chdir(par_ws)
!{cmd}
os.chdir(current_wd)

### Plot the parallel heads

In [ ]:
# build a single head dataset
model_names = list(new_sim.model_names)
head_tot_parallel = np.zeros((nlay, nrow, ncol), dtype=float)
im = 0
for idx in range(len(row_blocks)-1):
    for jdx in range(len(col_blocks)-1):
        m = new_sim.get_model(model_names[im])
        mhead = m.output.head().get_data()
        head_tot_parallel[:, row_blocks[idx]:row_blocks[idx+1], col_blocks[jdx]:col_blocks[jdx+1]] = mhead[:, :, :]
        # increment model number
        im += 1
wt_tot = flopy.utils.postprocessing.get_water_table(head_tot_parallel)

In [ ]:
plot_head2d(wt_tot)

### Check: parallel versus serial

In [ ]:
diff = head_tot_parallel - head_tot_serial
ilay = 1 # layer to plot
title = f"parallel minus multi-model (serial), layer {ilay+1}"
plot_diff(diff[ilay], title)

### Check: parallel versus single model

In [ ]:
diff = head_tot_parallel - gwfhead
ilay = 1
title = f"parallel minus single model, layer {ilay+1}"
plot_diff(diff[ilay], title)

bgf = flopy.mf6.utils.MfGrdFile("basin/model.dis.grb")

iaverts = bgf.modelgrid.iverts

ia, ja = bgf.ia - 1, bgf.ja - 1

bgf.ncells

ia[15400:15500]

ia.shape

adj_list = []
for i in range(ia.shape[0] - 1):
    ia0 = ia[i] + 1
    ia1 = ia[i + 1] - 1
    if ia1 == ia0:
        continue
    temp = [ja[ia[i]]]
    for j in range(ia0, ia1):
        temp.append(ja[j])
    adj_list.append(temp)
    
adj_list